In [1]:
from paddleocr import PaddleOCR,draw_ocr
from PIL import Image
from IPython.display import display
from llama_index.llms.ollama import Ollama

In [10]:
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log = False)
# EDIT ME: change the value of img_path to your file's location.
img_path = 'data/1996-388-035.pdf'
result = ocr.ocr(img_path, cls=True)
ocr_text = ''
for res in result:
    for line in res:
        ocr_text += (line[-1][0] + '\n')

print(ocr_text)

Santa Monica Police Officers
& Firefighters'Association
POST OFFICE BOX2160 SANTA MONICA,CALIFORNIA
90407-2160
Maps and Gov Info
RESERVE
Campaign Literature
CAMPAIGNS, POLITICAL-1996
MUNICIPAL, SANTA MONICA
GENERAL
Dear
Your police officers, paramedics, firefighters and our most unique City need your help on Tuesday,.
November 5th. Your commitment to vote insures that our elected officials will continue to follow the course
of action that you feel is in Santa Monica's best interest.
There have been a lot of positive changes over the past two years: Palisades Park, 3rd Street Promenade,
Lincoln Park, Santa Monica Pier and the homeless issues. It is important that we not slip backwards.You
can make an important difference in these policy making decisions by casting your vote for the council
candidates that are committed to keeping your safety and protection their top priorities..
Members of both the fire and police associations have interviewed the council candidates and we have
endorsed

In [3]:
from refined.data_types.base_types import Span, Entity
from refined.inference.processor import Refined

In [4]:
refined = Refined.from_pretrained(model_name='wikipedia_model',
                                  entity_set='wikipedia',
                                  use_precomputed_descriptions=False,
                                  download_files=True)

In [33]:
def construct_wikidata_url(id):
    return "https://www.wikidata.org/wiki/" + id
    
text = ocr_text
spans = refined.process_text(text)

named_entities = []
for span in spans:
    id = span.predicted_entity.wikidata_entity_id
    name = span.predicted_entity.wikipedia_entity_title

    if name is not None and id is not None:
        url = construct_wikidata_url(id)
        full_entity = (name_url)
        if full_entity not in named_entities:
            named_entities.append((name, url))
        



NameError: name 'name_url' is not defined

In [29]:
description_prompt = ("The below text is taken from a political advertisement."
    f" Using neutral language, summarize its contents. '''{ocr_text}'''"
    )
# title prompt needs work
title_prompt =  (" Using neutral language, provide a short title for this political advertisement in less than 10 words: "
                f"'''{ocr_text}'''")

llm = Ollama(base_url="http://ollama:11434", model="tinyllama", request_timeout=180.0)
description = llm.complete(description_prompt)
title = llm.complete(title_prompt)
print(description)
print()
print(title)

The contents of the political advertisement are a call to action for Santa Monica voters, urging them to cast their ballot on November 5th in favor of candidates committed to improving public safety and services. The text highlights the urgent need to ensure that Santa Monica's emergency communications system is properly functioning and evacuation procedures are in place, citing recent failures in these areas. The campaign is endorsed by various organizations, including firefighters and police officers, who insist on a "better City" for all residents. The text also highlights the need to vote in favor of the construction of a Public Safety Building, which has failed multiple times during earthquakes in Santa Monica, and the importance of ensuring that 911 emergency services are adequately funded and maintained. Overall, the message is urgent and calls on voters to support candidates committed to improving their safety and protection as Santa Monica continues to evolve into a thriving, 

In [32]:
import json
record_dict = {"title": str(title),
               "description": str(description),
               "related_entities": named_entities}

with open("output_record.json", "w") as outfile: 
    json.dump(record_dict, outfile)

